# <u>RNA-Seq Analysis Phase IIa: Alignment to Reference Genome</u>
## This Notebook illustrates how to align paired-end RNA-Seq reads <br>that have already been processed through the QC pipeline.
#### Last Revision: July 2017
#### Author: Charles David
## <span style="color:red"><u>Please use this Notebook in Conjunction with the RNA-Seq Best Practices Document:</u></span>
### <a href="https://github.com/PlantandFoodResearch/BestPractices/blob/bestpractices/_bestpractices/RNASeq/RNA-Seq-Best-Practice-Guidelines_REV_July_2017.md">RNA-Seq-Best-Practice-Guidelines_REV_July_2017.md</a>

## The Raw Data files are located on PowerPlant in the following location:
### `/input/genomic/Best_Practices_Testing/RNA_Seq/PE_Data`
## The Workflow is located on PowerPlant in the following location:
### `/workspace/Best_Practices_Testing/RNA_Seq/PE_Example`
#### As Always, Please Re-Create this Workflow in YOUR OWN Workspace.

## The Raw Data files are located on PowerPlant in the following location:


`/input/genomic/plant/Actinidia/chinensis/AGRF_CAGRF15208_CB6RHANXX`

Sequence name meaning
SRCT08_1: S=Simona; R=Red; CT=treatment name, control in this case; 08=weeks after anthesis, fruit age; 1=biological replicate
there are 4 sequencing files per each sample as they were run on 2 lanes (L001 and L002) and they were paired end (R1 and R2)

## <u>The Key Steps in this Section are: </u>##

#### I. Establish Data Management Structure on PowerPlant (Continuing from the QC part)
1. Make the necessary directories for the data and the analysis
2. Name the directories using standard workflow naming conventions
3. Name files using standard workflow naming conventions

#### II. Perform the Analyses: Align to Reference Genome Using the STAR Aligner
 1. Obtain reference genome and corresponding annotation file (GFF, GTF)
 2. Index the genome
 3. Align to genome:
    - Single-Pass Mode
    - Two-Pass Mode
 4. Merge uBAM and aligned BAM to produce "Clean" BAM
 5. Clean Up Workspace:
     - Delete un-needed intermediate files
     - Compress files that are still required

#### III. Assess the results for suitabillity in downstream analysis

## <u>Step I: Establish Data Management Structure on PowerPlant (Continuing from the QC part)</u>

### Create the analysis directories in YOUR workspace:
* 007.STAR
* 008.MBA

    * `mkdir /workspace/USER/RNA_Seq/PE_Example/007.STAR`
    * `mkdir /workspace/USER/RNA_Seq/PE_Example/007.STAR/logs`
    * `mkdir /workspace/USER/RNA_Seq/PE_Example/007.STAR/annotation`
    * `mkdir /workspace/USER/RNA_Seq/PE_Example/007.STAR/genome`
    * `mkdir /workspace/USER/RNA_Seq/PE_Example/007.STAR/index`
    * `mkdir /workspace/USER/RNA_Seq/PE_Example/007.STAR/index/logs`
    * `mkdir /workspace/USER/RNA_Seq/PE_Example/007.STAR/Single_Pass_Results`
    * `mkdir /workspace/USER/RNA_Seq/PE_Example/007.STAR/Two_Pass_Results`
    * `mkdir /workspace/USER/RNA_Seq/PE_Example/008.MBA`
    * `mkdir /workspace/USER/RNA_Seq/PE_Example/008.MBA/logs`
    * `mkdir /workspace/USER/RNA_Seq/PE_Example/008.MBA/Single_Pass_Results`
    * `mkdir /workspace/USER/RNA_Seq/PE_Example/008.MBA/Two_Pass_Results`


## <u>Step II, Part 1: Get the Genome and Annotation Files to be Used in the Alignment Process</u>
##### Note that best results are obtained if the reference is good quality and closely related, with annotations

### Define Project Variables:
* Note that we are using the latest version of STAR: 2.5.2b
* We are also using the latest version of Picard Tools: 2.9.4

In [1]:
# Define the user as a variable
USER="hradxj"
PROJECTNAME="karmun_awesome_experiment"
# Define the project directory and temp subdirectory as a variable
RAW=$PROJECT/000.raw
PROJECT="/workspace/$USER/$PROJECTNAME"
TEMP="$PROJECT/TEMP"
LOG=

TEMP="${PROJECT}/TEMP"
ANNOT="${PROJECT}/007.STAR/annotation/Niben101_annotation.gene_models.gff"
ZIPPEDGENOME="/workspace/ComparativeDataSources/Nicotiana/benthamiana/Genome/Niben.v1.0.1/assemblies/Niben.genome.v1.0.1.scaffolds.nrcontigs.fasta.gz"
INDEX="${PROJECT}/007.STAR/index"

PICARD="/workspace/cflcyd/software/picard/picard.jar"
LOG=$PROJECT/log
mkdir -p $LOG

In [3]:
echo $PROJECT

/workspace/hradxj/karmun_awesome_experiment


In [20]:
ls $PROJECT

000.raw		003.fastqc_smrna  006.MIA    008.MBA	  TEMP
001.fastqc_raw	004.trimmomatic   007a.STAR  009.STAR
002.SMRNA	005.fastqc_trim   007.STAR   Illumina.fa


#### Make appropriate directories and symlinks to files

In [107]:
mkdir ${PROJECT}/TEMP
mkdir ${PROJECT}/007.STAR
mkdir ${PROJECT}/007.STAR/logs
mkdir ${PROJECT}/007.STAR/annotation
mkdir ${PROJECT}/007.STAR/genome
mkdir ${PROJECT}/007.STAR/index
mkdir ${PROJECT}/007.STAR/index/logs
mkdir ${PROJECT}/007.STAR/Single_Pass_Results
mkdir ${PROJECT}/007.STAR/Two_Pass_Results
mkdir ${PROJECT}/008.MBA
mkdir ${PROJECT}/008.MBA/logs
mkdir ${PROJECT}/008.MBA/Single_Pass_Results
mkdir ${PROJECT}/008.MBA/Two_Pass_Results

ln -s  /workspace/ComparativeDataSources/Nicotiana/benthamiana/Genome/Niben.v1.0.1/annotation/Niben101/Niben101_annotation.gene_models.gff ${ANNOT}

mkdir: cannot create directory ‘/workspace/hradxj/karmun_awesome_experiment/TEMP’: File exists
mkdir: cannot create directory ‘/workspace/hradxj/karmun_awesome_experiment/008.MBA’: File exists
mkdir: cannot create directory ‘/workspace/hradxj/karmun_awesome_experiment/008.MBA/logs’: File exists
mkdir: cannot create directory ‘/workspace/hradxj/karmun_awesome_experiment/008.MBA/Single_Pass_Results’: File exists
mkdir: cannot create directory ‘/workspace/hradxj/karmun_awesome_experiment/008.MBA/Two_Pass_Results’: File exists


In [ ]:
# SANITY CHECK: have the directories and symlinks been made correctly?
ls -R ${PROJECT}

In [ ]:
# Obtain Grapevine leafroll-associated virus 3 and annotations
wget ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/851/885/GCF_000851885.1_ViralProj14906/GCF_000851885.1_ViralProj14906_genomic.fna.gz -O $RAW/GCF_000851885.1_ViralProj14906_genomic.fna.gz;
wget ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/851/885/GCF_000851885.1_ViralProj14906/GCF_000851885.1_ViralProj14906_genomic.gff.gz -O $RAW/GCF_000851885.1_ViralProj14906_genomic.gff.gz;
gunzip $RAW/*.gz;


## <u>Step II, Part 2: Index the Genome Using STAR</u>
* The inputs to this step are the genome as a multi FASTA file and the annotations as a GFF or GTF file
* The outputs include the genome index files used in the alignment steps

In [108]:
# The genome is a gzipped file and STAR requires unzipped files. Unzip

zcat ${ZIPPEDGENOME} > ${PROJECT}/007.STAR/genome/Niben.genome.v1.0.1.scaffolds.nrcontigs.fasta

In [10]:
GENOME=${PROJECT}/007.STAR/genome/Niben.genome.v1.0.1.scaffolds.nrcontigs.fasta;
echo $GENOME
echo $ANNOT

/workspace/hradxj/karmun_awesome_experiment/007.STAR/genome/Niben.genome.v1.0.1.scaffolds.nrcontigs.fasta
/workspace/hradxj/karmun_awesome_experiment/007.STAR/annotation/Niben101_annotation.gene_models.gff


In [110]:
# We need to examine the annotation file to set 3 parameters: the chromosome prefix and the text used
# to identify the parent-child relationship between exons and genes.
head -4 $ANNOT
# So the chromosome prefix is Niben101Ctg


Niben101Ctg00001	.	contig	1	500	.	.	.	ID=Niben101Ctg00001
Niben101Ctg00002	.	contig	1	500	.	.	.	ID=Niben101Ctg00002
Niben101Ctg00003	.	contig	1	500	.	.	.	ID=Niben101Ctg00003
Niben101Ctg00004	.	contig	1	500	.	.	.	ID=Niben101Ctg00004


In [111]:
# Let's check that the genome contains the right records
grep -A9 "Niben101Ctg00054" < $GENOME
# YES, this FASTA does contain a record exactly matching the annotation.

>Niben101Ctg00054 cov=9.0
AAAACCCAATTATTCTCTGAATCAATTCTCCTTCTTCCCTCTACCTCTCCTTTTCACTAA
AAACCTAAACATTTTTCAATATCTCTCTATTAACCCATTTATACATAAATCTACAACGCA
GTTCAGTTTGTTAAAGTTATTGCACTGTCTAAAAAAAAGAGCATCAATGGCTGAGCCAAC
AACACCAAACTCAGAATCAGAATCTAGTAGTTATAACTCTTGTTCTCTTTCTTCTACAAT
TTCATCTTCTTCTGTTCTTATAAAGAATATCAACTCGAAAAACCGACTCAAGAGATGCCG
TGAAGTAGCAGAAGAAAATGATGGTCAGACAAATATTAGTAAGTGCAGTAATAGCAGTAA
GAAAAATGGTGGCAACAAAACTAGCGACGGTTCAAAACACCCATCGTACGTTGGTGTACG
AAAGAGGGCATGGGGAAAATGGGTGTCCGAAATTCGTGAACCGAAGAAGAAATCAAGAAT
CTGGTTAGGTACTTTCGCCAC


From the [STAR manual](https://github.com/alexdobin/STAR/blob/master/doc/STARmanual.pdf):
>2.2.3 Annotations in GFF format.
In addition to the aforementioned options, for GFF3 formatted annotations you need to use
--sjdbGTFtagExonParentTranscript Parent. In general, for --sjdbGTFfile files STAR only
processes lines which have --sjdbGTFfeatureExon (=exon by default) in the 3rd field (column).
The exons are assigned to the transcripts using parent-child relationship defined by the
--sjdbGTFtagExonParentTranscript (=transcript id by default) GTF/GFF attribute.

In [112]:
# First let's look for the first line with "gene" in it
cat $ANNOT | grep gene | head -1
# We can see that the gene has an "ID=". This text identifies the gene. 
# The subcomponents of the gene will refer to this as a "parent"

Niben101Ctg00054	maker	gene	167	487	.	+	.	ID=Niben101Ctg00054g00001;Alias=snap_masked-Niben101Ctg00054-processed-gene-0.0
grep: write error
cat: write error: Broken pipe


In [113]:
# Let's examine all lines that contain this "ID=" tag.
grep "ID=Niben101Ctg00054g00001" < $ANNOT
# So the tag that identifies what transcript an exon comes from is "ID"
# and the tag that identifies what gene an mRNA or CDS comes from is also "ID" 


Niben101Ctg00054	maker	gene	167	487	.	+	.	ID=Niben101Ctg00054g00001;Alias=snap_masked-Niben101Ctg00054-processed-gene-0.0
Niben101Ctg00054	maker	mRNA	167	487	.	+	.	ID=Niben101Ctg00054g00001.1;Parent=Niben101Ctg00054g00001;Alias=snap_masked-Niben101Ctg00054-processed-gene-0.0-mRNA-1;Note="Ethylene-responsive transcription factor 7";Ontology_term=GO:0003677,GO:0006355,GO:0003700
Niben101Ctg00054	maker	exon	167	487	.	+	.	ID=Niben101Ctg00054g00001.1:exon:001;Parent=Niben101Ctg00054g00001.1;Alias=snap_masked-Niben101Ctg00054-processed-gene-0.0-mRNA-1:exon:4812
Niben101Ctg00054	maker	CDS	167	487	.	+	0	ID=Niben101Ctg00054g00001.1:cds:001;Parent=Niben101Ctg00054g00001.1;Alias=snap_masked-Niben101Ctg00054-processed-gene-0.0-mRNA-1:cds


Convert gff3 to gtf for use with STAR

In [56]:
/software/bioinformatics/cufflinks-2.2.1/gffread $ANNOT -T -o ${PROJECT}/007.STAR/annotation/Niben101_annotation.gene_models.gtf


In [ ]:
# For some reason, a single gene comes out with "geneID" instead of "gene_id"

In [83]:
grep geneID < $ANNOTGTF

Niben101Scf01064	maker	exon	568773	570384	.	+	.	transcript_id "Niben101Scf01064g05002.1"; geneID "Niben101Scf01064g05002";
Niben101Scf01064	maker	exon	570767	570807	.	+	.	transcript_id "Niben101Scf01064g05002.1"; geneID "Niben101Scf01064g05002";
Niben101Scf01064	maker	CDS	568773	570384	.	+	0	transcript_id "Niben101Scf01064g05002.1"; geneID "Niben101Scf01064g05002";
Niben101Scf01064	maker	CDS	570767	570807	.	+	2	transcript_id "Niben101Scf01064g05002.1"; geneID "Niben101Scf01064g05002";


In [80]:
# examine two genes to see what the difference is. g05002 comes out with geneID as a tag, g05007 comes out with gene_id as a tag.
grep "Niben101Scf01064g05002" < $ANNOT > test.gff
grep "Niben101Scf01064g05007" < $ANNOT >> test.gff;
cat test.gff

Niben101Scf01064	maker	gene	568773	570807	.	+	.	ID=Niben101Scf01064g05002;Alias=snap_masked-Niben101Scf01064-processed-gene-5.1
Niben101Scf01064	maker	exon	568773	570384	.	+	.	ID=Niben101Scf01064g05002.1:exon:001;Parent=Niben101Scf01064g05002.1;Alias=snap_masked-Niben101Scf01064-processed-gene-5.1-mRNA-1:exon:644
Niben101Scf01064	maker	exon	570767	570807	.	+	.	ID=Niben101Scf01064g05002.1:exon:002;Parent=Niben101Scf01064g05002.1;Alias=snap_masked-Niben101Scf01064-processed-gene-5.1-mRNA-1:exon:645
Niben101Scf01064	maker	CDS	568773	570384	.	+	0	ID=Niben101Scf01064g05002.1:cds:001;Parent=Niben101Scf01064g05002.1;Alias=snap_masked-Niben101Scf01064-processed-gene-5.1-mRNA-1:cds
Niben101Scf01064	maker	CDS	570767	570807	.	+	2	ID=Niben101Scf01064g05002.1:cds:002;Parent=Niben101Scf01064g05002.1;Alias=snap_masked-Niben101Scf01064-processed-gene-5.1-mRNA-1:cds
Niben101Scf01064	maker	gene	508219	513366	.	+	.	ID=Niben101Scf01064g05007;Alias=maker-Niben101Scf01064-snap-gene-5.6
Niben101Scf01064	make

In [81]:
# Convert the test.gff to gtf using cuffread to reproduce the problem
/software/bioinformatics/cufflinks-2.2.1/gffread test.gff -T -o-

Niben101Scf01064	maker	exon	508219	508245	.	+	.	transcript_id "Niben101Scf01064g05007.1"; gene_id "Niben101Scf01064g05007";
Niben101Scf01064	maker	exon	509505	509680	.	+	.	transcript_id "Niben101Scf01064g05007.1"; gene_id "Niben101Scf01064g05007";
Niben101Scf01064	maker	exon	511891	512049	.	+	.	transcript_id "Niben101Scf01064g05007.1"; gene_id "Niben101Scf01064g05007";
Niben101Scf01064	maker	exon	513075	513366	.	+	.	transcript_id "Niben101Scf01064g05007.1"; gene_id "Niben101Scf01064g05007";
Niben101Scf01064	maker	CDS	508219	508245	.	+	0	transcript_id "Niben101Scf01064g05007.1"; gene_id "Niben101Scf01064g05007";
Niben101Scf01064	maker	CDS	509505	509680	.	+	0	transcript_id "Niben101Scf01064g05007.1"; gene_id "Niben101Scf01064g05007";
Niben101Scf01064	maker	CDS	511891	512049	.	+	1	transcript_id "Niben101Scf01064g05007.1"; gene_id "Niben101Scf01064g05007";
Niben101Scf01064	maker	CDS	513075	513366	.	+	1	transcript_id "Niben101Scf01064g05007.1"; gene_id "Niben101Scf01064g05007";
Niben101Scf0

In [85]:
# I am unable to diagnose this problem. As a temporary solution, convert the gtf with a find-and-replace.
sed 's/geneID/gene_ID/g' < $ANNOTGTF > ${PROJECT}/007.STAR/annotation//Niben101_annotation.gene_models_fixedg05002.gtf;


In [3]:
# Reset the $ANNOTGTF variable
ANNOTGTF=${PROJECT}/007.STAR/annotation/Niben101_annotation.gene_models_fixedg05002.gtf

In [88]:
mkdir ${PROJECT}/007.STAR;
mkdir ${PROJECT}/007.STAR/logs;
mkdir ${PROJECT}/007.STAR/annotation;
mkdir ${PROJECT}/007.STAR/genome;
mkdir ${PROJECT}/007.STAR/index;
mkdir ${PROJECT}/007.STAR/index/logs;
mkdir ${PROJECT}/007.STAR/Single_Pass_Results;
mkdir ${PROJECT}/007.STAR/Two_Pass_Results;

GENOME=${PROJECT}/007.STAR/genome/Niben.genome.v1.0.1.scaffolds.nrcontigs.fasta;


COMMAND="module load STAR; \
STAR \
--runMode genomeGenerate \
--limitGenomeGenerateRAM 240000000000 \
--runThreadN 32 \
--genomeFastaFiles $GENOME \
--genomeDir ${PROJECT}/007.STAR/index \
--sjdbGTFfile ${ANNOTGTF} \
--sjdbGTFtagExonParentGene gene_id \
--sjdbGTFtagExonParentTranscript transcript_id; \
module unload STAR"
echo $COMMAND;
bsub \
-J STAR_Dan \
-o ${PROJECT}/007.STAR/index/logs/%J_STAR_index.out \
-e ${PROJECT}/007.STAR/index/logs/%J_STAR_index.err \
-n 32 \
$COMMAND
#-m aklppb34 \--sjdbGTFchrPrefix Niben101Ctg \

mkdir: cannot create directory ‘/workspace/hradxj/karmun_awesome_experiment/007.STAR’: File exists
mkdir: cannot create directory ‘/workspace/hradxj/karmun_awesome_experiment/007.STAR/logs’: File exists
mkdir: cannot create directory ‘/workspace/hradxj/karmun_awesome_experiment/007.STAR/annotation’: File exists
mkdir: cannot create directory ‘/workspace/hradxj/karmun_awesome_experiment/007.STAR/genome’: File exists
mkdir: cannot create directory ‘/workspace/hradxj/karmun_awesome_experiment/007.STAR/index’: File exists
mkdir: cannot create directory ‘/workspace/hradxj/karmun_awesome_experiment/007.STAR/index/logs’: File exists
mkdir: cannot create directory ‘/workspace/hradxj/karmun_awesome_experiment/007.STAR/Single_Pass_Results’: File exists
mkdir: cannot create directory ‘/workspace/hradxj/karmun_awesome_experiment/007.STAR/Two_Pass_Results’: File exists
module load STAR; STAR --runMode genomeGenerate --limitGenomeGenerateRAM 240000000000 --runThreadN 32 --genomeFastaFiles /workspace

In [89]:
bpeek -f 687778

<< output from stdout >>
Dec 06 13:41:42 ..... started STAR run
Dec 06 13:41:43 ... starting to generate Genome files
Dec 06 13:43:51 ... starting to sort Suffix Array. This may take a long time...
Dec 06 13:44:37 ... sorting Suffix Array chunks and saving them to disk...
Dec 06 13:56:53 ... loading chunks from disk, packing SA...
Dec 06 13:58:20 ... finished generating suffix array
Dec 06 13:58:20 ... generating Suffix Array index
Dec 06 14:03:13 ... completed Suffix Array index
Dec 06 14:03:13 ..... processing annotations GTF
Dec 06 14:03:19 ..... inserting junctions into the genome indices
Dec 06 14:05:42 ... writing Genome to disk ...
Dec 06 14:08:09 ... writing Suffix Array to disk ...
Dec 06 14:11:27 ... writing SAindex to disk
Dec 06 14:11:44 ..... finished successfully

------------------------------------------------------------
Sender: OpenLava System <openlava@wkoppb34>
Subject: Job 687778: <STAR_Dan> Done

Job <STAR_Dan> was submitted from host <aklppr31> by user <hradxj>.


In [90]:
#SANITY CHECK: did the index generation have any errors?
cat ${PROJECT}/007.STAR/index/logs/687778_STAR_index.err;


In [91]:
#SANITY CHECK: did the index generation work?
cat ${PROJECT}/007.STAR/index/logs/687778_STAR_index.out;


Dec 06 13:41:42 ..... started STAR run
Dec 06 13:41:43 ... starting to generate Genome files
Dec 06 13:43:51 ... starting to sort Suffix Array. This may take a long time...
Dec 06 13:44:37 ... sorting Suffix Array chunks and saving them to disk...
Dec 06 13:56:53 ... loading chunks from disk, packing SA...
Dec 06 13:58:20 ... finished generating suffix array
Dec 06 13:58:20 ... generating Suffix Array index
Dec 06 14:03:13 ... completed Suffix Array index
Dec 06 14:03:13 ..... processing annotations GTF
Dec 06 14:03:19 ..... inserting junctions into the genome indices
Dec 06 14:05:42 ... writing Genome to disk ...
Dec 06 14:08:09 ... writing Suffix Array to disk ...
Dec 06 14:11:27 ... writing SAindex to disk
Dec 06 14:11:44 ..... finished successfully

------------------------------------------------------------
Sender: OpenLava System <openlava@wkoppb34>
Subject: Job 687778: <STAR_Dan> Done

Job <STAR_Dan> was submitted from host <aklppr31> by user <hradxj>.
Job was executed on host(

In [92]:
ls $RAW

RACP005_11_S11_L002_R1.fastq.gz  RACP005_1_S8_L002_R1.fastq.gz
RACP005_11_S11_L002_R2.fastq.gz  RACP005_1_S8_L002_R2.fastq.gz
RACP005_12_S12_L002_R1.fastq.gz  RACP005_5_S9_L002_R1.fastq.gz
RACP005_12_S12_L002_R2.fastq.gz  RACP005_5_S9_L002_R2.fastq.gz
RACP005_13_S13_L002_R1.fastq.gz  RACP005_8_S10_L002_R1.fastq.gz
RACP005_13_S13_L002_R2.fastq.gz  RACP005_8_S10_L002_R2.fastq.gz


In [93]:
#SANITY CHECK:is there an index in the right place and is it a sensible size?
ls -s ${INDEX}

total 44620497
     546 chrLength.txt	          26 genomeParameters.txt
    1474 chrNameLength.txt         2 logs
    1218 chrName.txt	    24446922 SA
     914 chrStart.txt	     1822186 SAindex
   11946 exonGeTrInfo.tab       8042 sjdbInfo.txt
    4410 exonInfo.tab	        8722 sjdbList.fromGTF.out.tab
    1626 geneInfo.tab	        8722 sjdbList.out.tab
18299234 Genome		        4514 transcriptInfo.tab


In [95]:
#Find trimmed reads
ls ${PROJECT}/004.trimmomatic

logs
RACP005_11_S11_L002_MERGED_trimmomatic_R1.fastq
RACP005_11_S11_L002_MERGED_trimmomatic_R2.fastq
RACP005_12_S12_L002_MERGED_trimmomatic_R1.fastq
RACP005_12_S12_L002_MERGED_trimmomatic_R2.fastq
RACP005_13_S13_L002_MERGED_trimmomatic_R1.fastq
RACP005_13_S13_L002_MERGED_trimmomatic_R2.fastq
RACP005_1_S8_L002_MERGED_trimmomatic_R1.fastq
RACP005_1_S8_L002_MERGED_trimmomatic_R2.fastq
RACP005_5_S9_L002_MERGED_trimmomatic_R1.fastq
RACP005_5_S9_L002_MERGED_trimmomatic_R2.fastq
RACP005_8_S10_L002_MERGED_trimmomatic_R1.fastq
RACP005_8_S10_L002_MERGED_trimmomatic_R2.fastq
unpaired


In [117]:
TRIMMED=${PROJECT}/004.trimmomatic
OUT_STAR="${PROJECT}/007.STAR/Single_Pass_Results"
INDEX="${PROJECT}/007.STAR/index"
mkdir -p ${OUT_STAR}
PREFIXLIST=`basename -a ${TRIMMED}/*.fastq | sed 's/_R[1,2].fastq//g'|sort -u `
echo $PREFIXLIST

for PREFIX in ${PREFIXLIST}
do
echo $PREFIX
R1=${TRIMMED}/${PREFIX}_R1.fastq
R2=${TRIMMED}/${PREFIX}_R2.fastq


COMMAND="module load STAR; \
            STAR \
            --runThreadN 8 \
            --genomeDir ${INDEX} \
            --readFilesIn ${R1} ${R2} \
            --sjdbGTFfile ${ANNOTGTF} \
            --sjdbGTFtagExonParentGene gene_id \
            --sjdbGTFtagExonParentTranscript transcript_id \
            --outFileNamePrefix ${OUT_STAR}/${PREFIX}.bam \
            --quantMode TranscriptomeSAM GeneCounts \
            --outStd BAM_SortedByCoordinate"
echo $COMMAND
            
bsub \
-J STAR \
-o ${LOG}/%J_STAR_index.out \
-e ${LOG}/%J_STAR_index.err \
-n 8 \
-q lowpriority \
$COMMAND
done

RACP005_11_S11_L002_MERGED_trimmomatic RACP005_12_S12_L002_MERGED_trimmomatic RACP005_13_S13_L002_MERGED_trimmomatic RACP005_1_S8_L002_MERGED_trimmomatic RACP005_5_S9_L002_MERGED_trimmomatic RACP005_8_S10_L002_MERGED_trimmomatic
RACP005_11_S11_L002_MERGED_trimmomatic
module load STAR; STAR --runThreadN 8 --genomeDir /workspace/hradxj/karmun_awesome_experiment/007.STAR/index --readFilesIn /workspace/hradxj/karmun_awesome_experiment/004.trimmomatic/RACP005_11_S11_L002_MERGED_trimmomatic_R1.fastq /workspace/hradxj/karmun_awesome_experiment/004.trimmomatic/RACP005_11_S11_L002_MERGED_trimmomatic_R2.fastq --sjdbGTFfile /workspace/hradxj/karmun_awesome_experiment/007.STAR/annotation/Niben101_annotation.gene_models_fixedg05002.gtf --sjdbGTFtagExonParentGene gene_id --sjdbGTFtagExonParentTranscript transcript_id --outFileNamePrefix /workspace/hradxj/karmun_awesome_experiment/007.STAR/Single_Pass_Results/RACP005_11_S11_L002_MERGED_trimmomatic.bam --quantMode TranscriptomeSAM GeneCounts --outStd 

In [ ]:
bpeek -f 687902

<< output from stdout >>


In [121]:
# Run STAR in two-pass mode
# Create list of files containing splice junctions
SJLIST=$(ls ${PROJECT}/007.STAR/Single_Pass_Results/*SJ.out.tab)
echo $SJLIST

/workspace/hradxj/karmun_awesome_experiment/007.STAR/Single_Pass_Results/RACP005_11_S11_L002_MERGED_trimmomatic.bamSJ.out.tab /workspace/hradxj/karmun_awesome_experiment/007.STAR/Single_Pass_Results/RACP005_12_S12_L002_MERGED_trimmomatic.bamSJ.out.tab /workspace/hradxj/karmun_awesome_experiment/007.STAR/Single_Pass_Results/RACP005_13_S13_L002_MERGED_trimmomatic.bamSJ.out.tab /workspace/hradxj/karmun_awesome_experiment/007.STAR/Single_Pass_Results/RACP005_1_S8_L002_MERGED_trimmomatic.bamSJ.out.tab /workspace/hradxj/karmun_awesome_experiment/007.STAR/Single_Pass_Results/RACP005_5_S9_L002_MERGED_trimmomatic.bamSJ.out.tab /workspace/hradxj/karmun_awesome_experiment/007.STAR/Single_Pass_Results/RACP005_8_S10_L002_MERGED_trimmomatic.bamSJ.out.tab


In [122]:
TRIMMED=${PROJECT}/004.trimmomatic
INDEX=${PROJECT}/007.STAR/index
OUT_STAR=${PROJECT}/007.STAR/Two_Pass_Results
mkdir -p ${OUT_STAR}
PREFIXLIST=`basename -a ${TRIMMED}/*.fastq | sed 's/_R[1,2].fastq//g'|sort -u `
echo $PREFIXLIST

for PREFIX in ${PREFIXLIST}
do
echo $PREFIX
R1=${TRIMMED}/${PREFIX}_R1.fastq
R2=${TRIMMED}/${PREFIX}_R2.fastq


COMMAND="module load STAR; \
            STAR \
            --runThreadN 8 \
            --genomeDir ${INDEX} \
            --readFilesIn ${R1} ${R2} \
            --sjdbGTFfile ${ANNOTGTF} \
            --sjdbGTFtagExonParentGene gene_id \
            --sjdbGTFtagExonParentTranscript transcript_id \
            --outFileNamePrefix ${OUT_STAR}/${PREFIX}.bam \
            --sjdbFileChrStartEnd ${SJLIST} \
            --quantMode TranscriptomeSAM GeneCounts \
            --outStd BAM_SortedByCoordinate"
bsub \
-J STAR \
-o ${LOG}/%J_STAR_index.out \
-e ${LOG}/%J_STAR_index.err \
-n 8 \
-q lowpriority \
$COMMAND
done

RACP005_11_S11_L002_MERGED_trimmomatic RACP005_12_S12_L002_MERGED_trimmomatic RACP005_13_S13_L002_MERGED_trimmomatic RACP005_1_S8_L002_MERGED_trimmomatic RACP005_5_S9_L002_MERGED_trimmomatic RACP005_8_S10_L002_MERGED_trimmomatic
RACP005_11_S11_L002_MERGED_trimmomatic
Job <688654> is submitted to queue <lowpriority>.
RACP005_12_S12_L002_MERGED_trimmomatic
Job <688655> is submitted to queue <lowpriority>.
RACP005_13_S13_L002_MERGED_trimmomatic
Job <688656> is submitted to queue <lowpriority>.
RACP005_1_S8_L002_MERGED_trimmomatic
Job <688657> is submitted to queue <lowpriority>.
RACP005_5_S9_L002_MERGED_trimmomatic
Job <688658> is submitted to queue <lowpriority>.
RACP005_8_S10_L002_MERGED_trimmomatic
Job <688659> is submitted to queue <lowpriority>.


In [ ]:
bpeek -f 687038

<< output from stdout >>

------------------------------------------------------------
Sender: OpenLava System <openlava@aklppb32>
Subject: Job 687038: <STAR> Done

Job <STAR> was submitted from host <aklppr31> by user <hradxj>.
Job was executed on host(s) <2*aklppb32>, in queue <lowpriority>, as user <hradxj>.
                            <6*aklppb35>
                            <24*wkoppb37>
</home/hradxj> was used as the home directory.
</powerplant/workspace/hradxj/bioinf_karmun_awesome_experiment> was used as the working directory.
Started at Wed Dec  6 09:53:06 2017
Results reported at Wed Dec  6 10:44:41 2017

Your job looked like:

------------------------------------------------------------
# LSBATCH: User input
module load STAR; STAR --runThreadN 8 --genomeDir /workspace/hradxj/karmun_awesome_experiment/007.STAR/index --readFilesIn /workspace/hradxj/karmun_awesome_experiment/004.trimmomatic/RACP005_11_S11_L002_MERGED_trimmomatic_R1.fastq /workspace/hradxj/karmun_awesome_experi

In [3]:
# Analysis of BAM files (reads mapped)
ls $PROJECT


000.raw		003.fastqc_smrna  006.MIA   009.STAR	  10genes.txt  TEMP
001.fastqc_raw	004.trimmomatic   007.STAR  010.edgeR_Nb  Illumina.fa
002.SMRNA	005.fastqc_trim   008.MBA   011.edgeR_Vv  log


For this analysis, there are three samples:

RACP005_11_S11_L002                 Benth-Healthy-1
RACP005_12_S12_L002                 Benth-Healthy-2
RACP005_13_S13_L002                 Benth-Infected

Read counts have been generated by STAR. 
It is neccessary to create a single tab-delimited text file for import into R.

In this case, STAR produces a table of read counts that has the following characteristics:

1) The first four lines of the table contain information about the number of unmapped and multimapped reads.An example is shown below:

```
head RACP005_13_S13_L002_MERGED_trimmomatic.bamReadsPerGene.out.tab
N_unmapped      24827661        24827661        24827661
N_multimapping  6684546 6684546 6684546
N_noFeature     7429471 7663155 34125351
N_ambiguous     717277  256966  6802
```
These four lines are relevant but not used in the DE analysis, so should be removed.

2) The columns are ordered as follows

Gene name | Unstranded read counts | Sense strand read counts | Antisense strand read counts
--- | --- | --- | ---

We are interested in columns 1 and 3 (and column 4 if we choose to investigate antisense transcripts).

Therefore we need to create a file that contains the gene names in column 1, and the Sense strand read counts from 3 samples in columns 2,3,4.


In [125]:
# Find location of read count files
ls ${PROJECT}/007.STAR/Two_Pass_Results | grep ReadsPerGene

RACP005_11_S11_L002_MERGED_trimmomatic.bamReadsPerGene.out.tab
RACP005_12_S12_L002_MERGED_trimmomatic.bamReadsPerGene.out.tab
RACP005_13_S13_L002_MERGED_trimmomatic.bamReadsPerGene.out.tab


In [51]:
# First create a new file with one column of all gene names.

mkdir -p ${PROJECT}/010.edgeR_Nb;

cat ${PROJECT}/007.STAR/Two_Pass_Results/RACP005_11_S11_L002_MERGED_trimmomatic.bamReadsPerGene.out.tab |awk '{print $1}'\
> ${PROJECT}/010.edgeR_Nb/GRLaV3_Nb_EdgeR-genenames.tab;


In [52]:
# Now add the appropriate column
# Create a list of read count file names
READCOUNTFILELIST=$(ls ${PROJECT}/007.STAR/Two_Pass_Results | grep ReadsPerGene)


for READCOUNTFILE in $READCOUNTFILELIST
do
awk '{print $3}' < ${PROJECT}/007.STAR/Two_Pass_Results/${READCOUNTFILE} > ${PROJECT}/010.edgeR_Nb/${READCOUNTFILE}.col3;
done

paste ${PROJECT}/010.edgeR_Nb/GRLaV3_Nb_EdgeR-genenames.tab \
${PROJECT}/010.edgeR_Nb/RACP005_11_S11_L002_MERGED_trimmomatic.bamReadsPerGene.out.tab.col3 \
${PROJECT}/010.edgeR_Nb/RACP005_12_S12_L002_MERGED_trimmomatic.bamReadsPerGene.out.tab.col3 \
${PROJECT}/010.edgeR_Nb/RACP005_13_S13_L002_MERGED_trimmomatic.bamReadsPerGene.out.tab.col3 \
> ${PROJECT}/010.edgeR_Nb/GRLaV3_Nb_EdgeR_with_unmapped.tab


In [53]:
head ${PROJECT}/010.edgeR_Nb/GRLaV3_Nb_EdgeR_with_unmapped.tab


N_unmapped	11575247	1193884	24827661
N_multimapping	6457575	575648	6684546
N_noFeature	6314912	701699	7663155
N_ambiguous	219454	21609	256966
Niben101Ctg00054g00001	55	23	79
Niben101Ctg00074g00004	0	0	0
Niben101Ctg00116g00002	65	17	133
Niben101Ctg00129g00001	0	0	0
Niben101Ctg00141g00002	1	0	1
Niben101Ctg00174g00001	0	0	0


In [54]:
tail -59814  ${PROJECT}/010.edgeR_Nb/GRLaV3_Nb_EdgeR_with_unmapped.tab >  ${PROJECT}/010.edgeR_Nb/GRLaV3_Nb_EdgeR.tab


In [55]:
# Add header line

sed -i '1s/^/Gene\tBenth-Healthy-1\tBenth-Healthy-2\tBenth-Infected\n/' ${PROJECT}/010.edgeR_Nb/GRLaV3_Nb_EdgeR.tab;


In [56]:
# Check final formatting of file
head ${PROJECT}/010.edgeR_Nb/GRLaV3_Nb_EdgeR.tab
# We are now ready to analyse this in EdgeR

Gene	Benth-Healthy-1	Benth-Healthy-2	Benth-Infected
Niben101Ctg00054g00001	55	23	79
Niben101Ctg00074g00004	0	0	0
Niben101Ctg00116g00002	65	17	133
Niben101Ctg00129g00001	0	0	0
Niben101Ctg00141g00002	1	0	1
Niben101Ctg00174g00001	0	0	0
Niben101Ctg00192g00001	0	0	0
Niben101Ctg00219g00002	3	1	4
Niben101Ctg00228g00001	43	5	53


In [75]:
# We have derived a set of putatively differentially expressed genes. Examine the raw read counts as a sanity check

DEGENELIST="Niben101Scf00107g03008 Niben101Scf05044g02012 Niben101Scf00837g08001 Niben101Scf03937g00009 Niben101Scf06977g00014 Niben101Scf03169g00010 Niben101Scf01475g00019 Niben101Scf03506g03001 Niben101Scf02971g01006 Niben101Scf08683g00001"
echo $DEGENELIST



Niben101Scf00107g03008 Niben101Scf05044g02012 Niben101Scf00837g08001 Niben101Scf03937g00009 Niben101Scf06977g00014 Niben101Scf03169g00010 Niben101Scf01475g00019 Niben101Scf03506g03001 Niben101Scf02971g01006 Niben101Scf08683g00001


In [76]:
for DEGENE in $DEGENELIST
do
grep $DEGENE < ${PROJECT}/007.STAR/Two_Pass_Results/RACP005_11_S11_L002_MERGED_trimmomatic.bamReadsPerGene.out.tab | awk '{print $1}'
echo "Benth-healthy-1"
grep $DEGENE < ${PROJECT}/007.STAR/Two_Pass_Results/RACP005_11_S11_L002_MERGED_trimmomatic.bamReadsPerGene.out.tab | awk '{print $3}'
echo "Benth-healthy-2"
grep $DEGENE < ${PROJECT}/007.STAR/Two_Pass_Results/RACP005_12_S12_L002_MERGED_trimmomatic.bamReadsPerGene.out.tab | awk '{print $3}'
echo "Benth-infected"
grep $DEGENE < ${PROJECT}/007.STAR/Two_Pass_Results/RACP005_13_S13_L002_MERGED_trimmomatic.bamReadsPerGene.out.tab | awk '{print $3}'

done

Niben101Scf00107g03008
Benth-healthy-1
10
Benth-healthy-2
3
Benth-infected
2328
Niben101Scf05044g02012
Benth-healthy-1
3
Benth-healthy-2
1
Benth-infected
581
Niben101Scf00837g08001
Benth-healthy-1
1053
Benth-healthy-2
7
Benth-infected
7
Niben101Scf03937g00009
Benth-healthy-1
11
Benth-healthy-2
6
Benth-infected
1150
Niben101Scf06977g00014
Benth-healthy-1
5068
Benth-healthy-2
23
Benth-infected
40
Niben101Scf03169g00010
Benth-healthy-1
1192
Benth-healthy-2
6
Benth-infected
15
Niben101Scf01475g00019
Benth-healthy-1
696
Benth-healthy-2
0
Benth-infected
8
Niben101Scf03506g03001
Benth-healthy-1
24
Benth-healthy-2
2
Benth-infected
777
Niben101Scf02971g01006
Benth-healthy-1
162
Benth-healthy-2
9
Benth-infected
0
Niben101Scf08683g00001
Benth-healthy-1
1149
Benth-healthy-2
7
Benth-infected
22
